# 0 - Library

In [1]:
import cv2 
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

# 1 - Background Capturing

In [2]:
def countdown_capture(cam_id=0, t=5,  cam_width=1280, cam_height =720, shrink_size=200,):
    
    '''
    cam_id = camera device id (default = 0 (built-in camera)) 
    t = time to  count down (default = 5 second).
    cam_width = camera width range (default = 1280 pixel, HD reslution).
    cam_height = camera height range (default = 720 pixel, HD resolution).
    shrink_size = Shrink the width of image (default = 200 pixel).
    '''

    cap = cv2.VideoCapture(cam_id, cv2.CAP_DSHOW) #  0 [default] = your main camera, 1 = my usb camera (cv2.CAP_DSHOW = DIRECT show)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, cam_width) # HD resolution setting
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cam_height) # HD resolution setting
    
    
    while(t>=0):

        _, bg = cap.read() # bg = captured background in an array of pixels 
        bg = np.flip(bg, axis=1) # horizontal flip
        bg = bg[:,shrink_size:cam_width-shrink_size,:] # shrink image
        
        # Display timer on Frame or Background (bg)
        bg_text = cv2.putText(bg.copy(), text=str(t), org=(bg.shape[0]//2, bg.shape[1]//2), 
                              fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=8, color = (255, 0, 255), 
                              thickness=10, lineType= cv2.LINE_AA)
        
        
        # show result
        (win_width, win_height) = (640, 480)
        cv2.namedWindow('Background Image', cv2.WINDOW_NORMAL)
        cv2.imshow('Background Image', bg_text)
        cv2.resizeWindow('Background Image', win_width, win_height)

        # the 'q' button is set as the quitting button you may use any desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break


        time.sleep(1) # sleep 1 second
        t-=1 # t=t-1
        
        
    cap.release() # After the loop release the cap object
    cv2.destroyAllWindows() # Destroy all the windows
    
    
    return bg

In [3]:
# bg = countdown_capture(t=7)

# 2 - Cloak Detection and Background Replacement

- LAB Image Conversion
- Image Thresholding
- Countour Finding
- Noise Filtering
- Morphological Operation
- Logical Operation

## LAB Color space

<img src="preview.jpg">

In [4]:
def invisibility_cloak(bg, cam_id=0, cam_width=1280, cam_height =720, shrink_size=200, save_video=False):
    
    '''
    bg = background
    cam_id = camera device id (default = 0 (built-in camera)) 
    cam_width = camera width range (default = 1280 pixel, HD reslution).
    cam_height = camera height range (default = 720 pixel, HD resolution).
    shrink_size = Shrink the width of image (default = 200 pixel).
    save_video == would you like to save a video of your result (default  = False)  
    '''

    cap = cv2.VideoCapture(cam, cv2.CAP_DSHOW) #  0 [default] = your main camera, 1 = my usb camera (cv2.CAP_DSHOW = DIRECT show)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, cam_width) # HD resolution setting
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cam_height) # HD resolution setting

    if save_video:
        fourcc = cv2.VideoWriter_fourcc(*'MP4V')
        inputt = cv2.VideoWriter('input.mp4', fourcc, 12.0, (cam_width-2*shrink_size, cam_height))
        result = cv2.VideoWriter('output.mp4', fourcc, 12.0, (cam_width-2*shrink_size, cam_height))
        
    
    while(cap.isOpened()):
        
        
        _, frame = cap.read() # get image frame
        frame = np.flip(frame, axis=1) # horizontal flip
        frame = frame[:,shrink_size:cam_width-shrink_size,:] # fixed range
        
        ####################################### LAB Image Conversion #######################################
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB) # convert BGR image to LAB color space
        a_channel = lab[:, :, 1] # green channel
        
        ####################################### Image Thresholding #########################################
        blur = cv2.blur(a_channel, (7,7)) # blur image 
        threshold_img = cv2.threshold(blur, 115, 255, cv2.THRESH_BINARY_INV)[1]  # binary thresholding
        
        ####################################### Contour Finding ###########################################
        contours, _ = cv2.findContours(threshold_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        ####################################### Noise Filtering ###########################################
        h, w = frame.shape[:2] # get frame shape
        mask = np.zeros((h, w), np.uint8)  # create black mask
        
        for cnt in contours:
            area = cv2.contourArea(cnt) # Find area (#pixel) of each contour
            if area>700: # choose object with area >700
                cv2.drawContours(mask, [cnt], -1, 255, -1) # draw mask
                
        
        ####################################### Morphological Operation ###################################
        kernel = np.ones((3, 3), np.uint8) # create kernel
        mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel, iterations=4) # dilate (idea:to enlarge our mask to cover the border
        
        ####################################### Logical Operation ########################################
        inv_mask = cv2.bitwise_not(mask) # get inversed mask (mask outside cloak)
        outside_cloak = cv2.bitwise_and(frame, frame, mask=np.uint8(inv_mask/255))
        inside_cloak = cv2.bitwise_and(bg, bg, mask=np.uint8(mask/255))
        output = cv2.addWeighted(src1=outside_cloak, alpha=1, src2=inside_cloak, beta=1, gamma=0) # new_image = alpha * src1 + beta * src2 + gamma
        
        
        if save_video:
            inputt.write(frame)
            result.write(output)
            

        
        # Display the resulting frame
        (win_width, win_height) = (320,240)

        cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)
        cv2.imshow('Frame', frame)
        cv2.resizeWindow('Frame', win_width, win_height)

        cv2.namedWindow('LAB Colorspace (a_channel)', cv2.WINDOW_NORMAL)
        cv2.imshow('LAB Colorspace (a_channel)', a_channel)
        cv2.resizeWindow('LAB Colorspace (a_channel)', win_width, win_height)

        # cv2.namedWindow('Blur Image', cv2.WINDOW_NORMAL)
        # cv2.imshow('Blur Image', blur)
        # cv2.resizeWindow('Blur Image', win_width, win_height)

        cv2.namedWindow('Thresholding Image', cv2.WINDOW_NORMAL)
        cv2.imshow('Thresholding Image', threshold_img)
        cv2.resizeWindow('Thresholding Image', win_width, win_height)

        cv2.namedWindow('Mask (Noise Filtering)', cv2.WINDOW_NORMAL)
        cv2.imshow('Mask (Noise Filtering)', mask)
        cv2.resizeWindow('Mask (Noise Filtering)', win_width, win_height)

        cv2.namedWindow('Final Output', cv2.WINDOW_NORMAL)
        cv2.imshow('Final Output', output)
        cv2.resizeWindow('Final Output', win_width, win_height)
        

        # the 'q' button is set as the quitting button you may use any desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        
    cap.release() # After the loop release the cap object
    
    if save_video:
        inputt.release()
        result.release()

        
    cv2.destroyAllWindows() # Destroy all the windows

In [39]:
# def invisibility_cloak2(bg, cam=0, use_cam=True, shrink_size=200, cam_width=1280, cam_height =720, save_video=False):
    
#     '''
#     cam = camera id or "Link of video" 
#     use_camera (default = True (using)), False (not-using)
#     shrink_size = Shrink the width of image (default = 200 pixel).
#     cam_width = camera width range (default = 1280 pixel, HD reslution).
#     cam_height = camera height range (default = 720 pixel, HD resolution).
#     save_video == would you like to save a video of your result (default  = False)  
#     '''
    
#     if use_cam:
#         cap = cv2.VideoCapture(cam, cv2.CAP_DSHOW) #  0 [default] = your main camera, 1 = my usb camera (cv2.CAP_DSHOW = DIRECT show)
#         cap.set(cv2.CAP_PROP_FRAME_WIDTH, cam_width) # HD resolution setting
#         cap.set(cv2.CAP_PROP_FRAME_HEIGHT, cam_height) # HD resolution setting
#     else:
#         cap = cv2.VideoCapture(cam)
#         fourcc = cv2.VideoWriter_fourcc(*'MP4V')
#         inputt = cv2.VideoWriter('inputfromlink.mp4', fourcc, 60.0, (cam_width, cam_height))
#         result = cv2.VideoWriter('outputfromlink.mp4', fourcc, 60.0, (cam_width, cam_height))
    
#     if save_video:
#         fourcc = cv2.VideoWriter_fourcc(*'MP4V')
#         inputt = cv2.VideoWriter('input.mp4', fourcc, 12.0, (cam_width-2*shrink_size, cam_height))
#         result = cv2.VideoWriter('output.mp4', fourcc, 12.0, (cam_width-2*shrink_size, cam_height))
        
    
#     while(cap.isOpened()):
        
        
#         ret, frame = cap.read() # get image frame
#         # print(~ret)
        
#         if ret==False:
#             break
        
#         if use_cam:
#             frame = np.flip(frame, axis=1) # horizontal flip
#             frame = frame[:,shrink_size:cam_width-shrink_size,:] # fixed range
            
        
#         # RGB -> LAB color space 
#         lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB) # convert BGR image to LAB color space
#         a_channel = lab[:, :, 1] # green channel
        
#         # Thresholding
#         blur = cv2.blur(a_channel, (7,7)) # blur image by local mean 
#         threshold_img = cv2.threshold(blur, 115, 255, cv2.THRESH_BINARY_INV)[1]  # binary thresholding
        
#         # Find Contours
#         contours, _ = cv2.findContours(threshold_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
#         # Noise Filtering
#         h, w = frame.shape[:2] # get frame shape
#         mask = np.zeros((h, w), np.uint8)  # create black mask
        
#         for cnt in contours:
#             area = cv2.contourArea(cnt) # Find area (#pixel) of each contour
#             if area>700:
#                 cv2.drawContours(mask, [cnt], -1, 255, -1) # draw mask
                
        
#         # Morphological Operation
#         kernel = np.ones((3, 3), np.uint8) # create kernel
#         mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel, iterations=4) # dilate (idea:to enlarge our mask to cover the border
        
#         # Logical Operation
#         inv_mask = cv2.bitwise_not(mask) # get inversed mask (mask outside cloak)
#         outside_cloak = cv2.bitwise_and(frame, frame, mask=np.uint8(inv_mask/255))
#         inside_cloak = cv2.bitwise_and(bg, bg, mask=np.uint8(mask/255))
#         output = cv2.addWeighted(src1=outside_cloak, alpha=1, src2=inside_cloak, beta=1, gamma=0) # new_image = alpha * src1 + beta * src2 + gamma
        
        
#         if save_video:
#             inputt.write(frame)
#             result.write(output)
            
#         if ~use_cam:
#             inputt.write(frame)
#             result.write(output)
        
        
#         # Display the resulting frame
#         (win_width, win_height) = (320,240)

#         cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)
#         cv2.imshow('Frame', frame)
#         cv2.resizeWindow('Frame', win_width, win_height)

#         cv2.namedWindow('LAB Colorspace (a_channel)', cv2.WINDOW_NORMAL)
#         cv2.imshow('LAB Colorspace (a_channel)', a_channel)
#         cv2.resizeWindow('LAB Colorspace (a_channel)', win_width, win_height)

#         # cv2.namedWindow('Blur Image', cv2.WINDOW_NORMAL)
#         # cv2.imshow('Blur Image', blur)
#         # cv2.resizeWindow('Blur Image', win_width, win_height)

#         cv2.namedWindow('Thresholding Image', cv2.WINDOW_NORMAL)
#         cv2.imshow('Thresholding Image', threshold_img)
#         cv2.resizeWindow('Thresholding Image', win_width, win_height)

#         cv2.namedWindow('Mask (Noise Filtering)', cv2.WINDOW_NORMAL)
#         cv2.imshow('Mask (Noise Filtering)', mask)
#         cv2.resizeWindow('Mask (Noise Filtering)', win_width, win_height)

#         cv2.namedWindow('Final Output', cv2.WINDOW_NORMAL)
#         cv2.imshow('Final Output', output)
#         cv2.resizeWindow('Final Output', win_width, win_height)
        

#         # the 'q' button is set as the quitting button you may use any desired button of your choice
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
        
        
#     cap.release() # After the loop release the cap object
    
#     if save_video:
#         inputt.release()
#         result.release()
#     if ~use_cam:
#         inputt.release()
#         result.release()
        
#     cv2.destroyAllWindows() # Destroy all the windows

# Run

In [5]:
if __name__ == '__main__':
    bg = countdown_capture()
    invisibility_cloak(bg, save_video=True

In [ ]:
# if __name__ == '__main__':
#     # bg = countdown_capture()
#     invisibility_cloak2(bg, cam="IMG_2534.MOV", use_cam=False , save_video=False, cam_width=1920, cam_height =1080)